# 2.4 Named Graphs

Named Graphs offer a way to partition your data and are commonly used to group semantically similar data.

Data in named graphs is stored as quads, with the fourth entry representing the named graph.

Rules can reference several named graphs in both the head and body simultaneously.

## The default graph

The default graph is unlike named graphs in that it has no name and stores triples rather than quads.

Without specify a named graph RDFox will reference the default graph.

### Tuples tables

RDFox actually stores data as lists of facts in in-memory containers called tuple tables.

Two tuple tables are created automatically upon creating a new data store, the `DefaultTriples` table that stores triples in the default graph and the `Quads` table that contains all named graphs quads.

Alongside others, such as a name, a defining feature of tuple tables are is their 'arity' - the number of columns they contain.

Additional tuple tables can be created of arity 1-4 (with some built-in exceptions).

One such tuple table is the **SKOLEM** table which can have arity from one upwards. Here, the last entry in the row is a blank node that is uniquely determined by all of the preceding arguments. 

Tuple tables, including those automatically created, can be deleted to save memory if required.

## Example

The following example shows how the Default Graph and a Named Graph can be used in a single rule.

The Named Graph ':Personnel' contains information about staff for a financial institute and the Default Graph contains in formation about trades that have been executed.

The rules infers information about the staff who are active traders.

In [71]:
ng_data = """

@prefix : <https://rdfox.com/example#> .

:PersonnelInfo {
    :p-001 a :StaffMember .
    :p-002 a :StaffMember .
}

:t-001001 a :Transaction ;
    :executedBy :p-001 .

"""

In [72]:
ng_rules = """

[?personnel, a, :Trader] :PersonnelInfo :-
    [?personnel, a, :StaffMember] :PersonnelInfo,
    [?trade, :executedBy, ?personnel] .

"""

In [73]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=ng_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=ng_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Get and issue select query
with open("../queries/2_4-NamedGraphsQuery.rq", "r") as file:
    ng_query = file.read()
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": ng_query})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Personnel Rolls ===')
print(response.text)


=== Personnel Rolls ===
?personnel	?roll
<https://rdfox.com/example#p-00001>	<https://rdfox.com/example#Trader>
<https://rdfox.com/example#p-00002>	<https://rdfox.com/example#StaffMember>



## Named Graphs syntax

As you can see in the rule above, named graphs can be accessed by including its name (or inserting a variable) after the desired triple pattern within the graph.

`[?S, ?O, ?O] ?G`

A slightly different syntax is used when writing queries, declaring the Named Graph before the triple pattern with the `GRAPH` keyword.

`GRAPH ?G {?S ?P ?O} `

### Tuple table syntax

Accessing custom tuple tables is slightly different again, with rule atoms taking the form:

`TupleTableName(?S, ?P, ?O, ?Q, ...)`

and queries accessing them via the `TT` keyword:

`TT TupleTableName { ?S ?P ?O ?Q ...}`

## Exercise

Complete the rule `2_4-NamedGraphsRules.dlog` in the `rules` folder so that the query below can be used to directly find traders who have breached their limits.

### Helpful resources

[Named Graphs syntax in RDFox](https://oxfordsemtech.github.io/DocumentationDevBuild/reasoning.html#named-graphs-and-n-ary-relations)

In [98]:
ng_sparql = """

SELECT ?personnel ?limit ?trade ?tradeValue
WHERE {
    GRAPH :PersonnelInfo {
        ?personnel :inBreachOfRegulation true ;
            :hasMaxTradeRestriction ?limit.
    } .

    ?trade :executedBy ?personnel ;
        :hasValue ?tradeValue ;
        a :BeachOfRegulation .
}

"""

Here is a representative sample of the data in `2_4-NamedGraphsData.ttl`.

In [81]:
sample_data = """
@prefix : <https://rdfox.com/example#> .

:PersonnelInfo {
    :p-001 a :Trader ;
        :hasMaxTradeRestriction 100000 .
    
    :p-002 a :StaffMember .
}

:t-000001 a :Transaction ;
    :executedBy :p-001 ;
    :hasValue 50000 .

"""

### Check your work

Run the query below to verify the results.

In [99]:
# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Get and add data
with open("../data/2_4-NamedGraphsData.ttl", "r") as file:
    namedGraphs_data = file.read()
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=namedGraphs_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get and add rules
with open("../rules/2_4-NamedGraphsRules.dlog", "r") as file:
    namedGraph_rules = file.read()
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=namedGraph_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Issue select query
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": ng_sparql})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Traders in Trouble ===')
print(response.text)


=== Traders in Trouble ===
?personnel	?limit	?trade	?tradeValue
<https://rdfox.com/example#p-075>	80000	<https://rdfox.com/example#t-075011>	115606
<https://rdfox.com/example#p-041>	130000	<https://rdfox.com/example#t-041021>	185874
<https://rdfox.com/example#p-043>	150000	<https://rdfox.com/example#t-043025>	222928



### You should see...

=== Traders in Trouble ===
|?personnel|?limit|?trade|?tradeValue|
|-----------|-------------|-------------|-------------|
|<https://rdfox.com/example#p-075>|	80000|	<https://rdfox.com/example#t-075011>|	115606|
|<https://rdfox.com/example#p-041>|	130000|	<https://rdfox.com/example#t-041021>|	185874|
|<https://rdfox.com/example#p-043>|	150000|	<https://rdfox.com/example#t-043025>|	222928|